In [ ]:
!pip install mri-nufft[cufinufft,finufft,gpunufft,extra,autodiff]
!pip install brainweb-dl fastmri


# Trajectory constraints

A collection of methods to make trajectories fit hardware constraints.


Hereafter we illustrate different methods to reduce the gradient
strengths and slew rates required for the trajectory to match the
hardware constraints of MRI machines. A summary table is available
below.




.. list-table:: Constraint fitting methods
   :header-rows: 1

   * -
     - Gradient strength
     - Slew rate
     - Path preserved
     - Density preserved
   * - Arc-length parameterization
     - Yes
     - No
     - Yes
     - No
  * - Projection onto convex sets
     - Yes
     - Yes
     - No
     - No




In [ ]:
# Internal
import mrinufft as mn
from mrinufft.trajectories.utils import Acquisition, compute_gradients_and_slew_rates
from utils import show_trajectory_full

# External
import numpy as np

## Script options
These options are used in the examples below as default values for all trajectories.



In [ ]:
# Acquisition parameters
acq = Acquisition.default

In [ ]:
# Trajectory parameters
Nc = 4  # Number of shots
Ns = 512  # Number of samples per shot
in_out = False  # Choose between in-out or center-out trajectories
nb_revolutions = 4  # Number of zigzags for base trajectories

In [ ]:
# Display parameters
figure_size = 10  # Figure size for trajectory plots
subfigure_size = 6  # Figure size for subplots
one_shot = 2 * Nc // 3  # Highlight one shot in particular
sample_freq = 60  # Frequency of samples to display in the trajectory plots

We will be using a cone trajectory to showcase the different methods as
it switches several times between high gradients and slew rates.



In [ ]:
original_trajectory = mn.initialize_2D_spiral(
    Nc, Ns, in_out=in_out, nb_revolutions=nb_revolutions
)

show_trajectory_full(
    original_trajectory, one_shot, subfigure_size, sample_freq, acq=acq
)

grads, slews = compute_gradients_and_slew_rates(original_trajectory, acq)
grad_max, slew_max = np.max(grads), np.max(slews)
print(f"Max gradient: {grad_max:.3f} T/m, Max slew rate: {slew_max:.3f} T/m/ms")

## Arc-length parameterization
Arc-length parameterization is the simplest method to reduce the gradient
strength as it resamples the trajectory to have a constant distance between
samples. This is technically the lowest gradient strength achievable while
preserving the path of the trajectory, but it does not preserve the k-space
density and can lead to high slew rates as shown below.



In [ ]:
from mrinufft.trajectories.projection import parameterize_by_arc_length

projected_trajectory = parameterize_by_arc_length(original_trajectory)

In [ ]:
show_trajectory_full(
    projected_trajectory, one_shot, subfigure_size, sample_freq, acq=acq
)

grads, slews = compute_gradients_and_slew_rates(projected_trajectory, acq)
grad_max, slew_max = np.max(grads), np.max(slews)
print(f"Max gradient: {grad_max:.3f} T/m, Max slew rate: {slew_max:.3f} T/m/ms")

## Projection onto convex sets
The projection step is a primal-dual algorithm to project any trajectory
into the convex constraint set. This step guarantees that the final trajectory
is playably by the scanner. Also, as the constraint set if convex, the
projection results in unique trajectory which is closest to the original
one while being hardware compliant.



In [ ]:
from mrinufft.trajectories.projection import project_trajectory

projected_trajectory = project_trajectory(
    original_trajectory,
    acq,
    max_iter=10000,
    TE_pos=0,
)

In [ ]:
show_trajectory_full(
    projected_trajectory, one_shot, subfigure_size, sample_freq, acq=acq
)
grads, slews = compute_gradients_and_slew_rates(projected_trajectory, acq)
grad_max, slew_max = np.max(grads), np.max(slews)
print(f"Max gradient: {grad_max:.3f} T/m, Max slew rate: {slew_max:.3f} T/m/ms")